# [1. Maximum Length of Repeated Subarray](https://leetcode.com/problems/maximum-length-of-repeated-subarray/)

- **Given**
    + 2 arrays:
        + `A` size N
        + `B` size M
- **Find**: **maximum length of a subarray** that appears in both arrays

#### Example 1

```
Input: A = [1,2,3,2,1], B = [3,2,1,4,7]
Output: 3
Explanation: The repeated subarray with maximum length is [3,2,1].
```

#### Example 2

```
Input: A = [0,0,0,0,0], B = [0,0,0,0,0]
Output: 5
```

## Solution
- Top down DP $O(N^2)$

```Cpp
class Solution {
public:
    int findLength(vector<int> &A, vector<int> &B) {
        int n = A.size();
        int m = B.size();
        int ans = 0;

        // dp[i][j]: the longest common substring of the prefixes: A[:i] and B[:j]
        vector<vector<int>> dp(n+1, vector<int>(m+1, 0));

        // Update from left to right
        for(int i=0; i<n; ++i) for(int j=0; j<m; ++j) {
            // Case: continous-common element
            if (i>0 && j>0 && A[i] == B[j] && A[i-1] == B[j-1]) {
                dp[i][j] = dp[i-1][j-1] + 1;
            }
            // Case: starting of a common subarray
            else if (A[i] == B[j]) {
                dp[i][j] = 1;
            }

            // Relax
            ans = max(ans, dp[i][j]);
        }

        return ans;
    }
};
```

#### Optimized Space
- Since dp update only requires `i` and `i-1` -> reduce the first dimension to 2

```C++
class Solution {
public:
    int findLength(vector<int> &A, vector<int> &B) {
        int n = A.size();
        int m = B.size();
        int ans = 0;

        // dp[i][j]: the longest common substring of the prefixes: A[:i] and B[:j]
        vector<vector<int>> dp(2, vector<int>(m+1, 0));

        // Update from left to right
        for(int i=0; i<n; ++i) for(int j=0; j<m; ++j) {
            // Case: continous common elements
            if (i>0 && j>0 && A[i] == B[j] && A[i-1] == B[j-1]) {
                dp[i%2][j] = dp[(i+1)%2][j-1] + 1;
            }
            // Case: starting a common subarray
            else if (A[i] == B[j]) {
                dp[i%2][j] = 1;
            }

            // Relax
            ans = max(ans, dp[i%2][j]);
        }

        return ans;
    }
};
````


# 2. Maximum Length of Repeated Subarray, return ans
- **Given**
    + visiting history of `user1`
    + visiting history of `user2`
- **Find**
    + **Longest continuous common** history between two users
    
#### Example

```c++
Input:
    vector<string> user1 = {"3234.html", "xys.html", "7hsaa.html"};
    vector<string> user2 = {"3234.html", "sdhsfjdsh.html", "xys.html", "7hsaa.html"};
    
Output: {"xys.html", "7hsaa.html"}
```

## Solution
- binsearch by len: $O( \text{log(min(N,M))} * (N+M) * \text{min(N,M)}  )$

```Cpp
class Solution {
private:
    vector<string> _A, _B;
    int _N, _M;
    string __concat_string(vector<string> &s, int st, int en) {
        // Return the hash string (3234.html-sdhsfjdsh.html-)
        string res = "";
        for(int i=st; i<en; ++i) res += s[i] + "-";
        return res;
    }
    vector<string> __get_common_by_len(int len) {
        unordered_set<string> table;

        // Build table with A
        for(int i=0; i+len-1 < _N; ++i) {
            table.insert( __concat_string(_A, i, i+len) );
        }

        // Check common substring with B
        for(int j=0; j+len-1 < _M; ++j) {
            string tmp = __concat_string(_B, j, j+len);
            // If match return the sub array size len
            if(table.count(tmp)) return vector<string>(_B.begin() + j, _B.begin() + j + len);
        }

        // No match return array size 0
        return vector<string>();
    }
public:
    vector<string> longestCommonHistory(vector<string> &user1, vector<string> &user2) {
        // Init
        _A = user1;
        _N = _A.size();

        _B = user2;
        _M = _B.size();

        // Bin search for optimal length mid
        int l = 0;
        int r = min(_N, _M);

        vector<string> ans = {};
        while(l < r) {
            // m = optimal sub array length
            int m = l + (r-l)/2;
            // tmp: common sub array, length = m
            vector<string> tmp = __get_common_by_len(m);

            // search right: increase len
            if(tmp.size() > 0) {
                l = m + 1;

                // relax ans
                if(tmp.size() > ans.size()) ans = tmp;
            }

            // search left: decrease len
            else {
                r = m - 1;
            }
        }

        return ans;
    }
};
```